# Seq2seq
![img](arch.png)

# Teacher forcing
![img](teacher_forcing.png)

In [1]:
!head rus.txt

Go.	Иди.
Go.	Идите.
Hi.	Здравствуйте.
Hi.	Привет!
Hi.	Хай.
Hi.	Здрасте.
Hi.	Здоро́во!
Run!	Беги!
Run!	Бегите!
Run.	Беги!


In [2]:
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
import numpy as np

/opt/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
num_samples = 50000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'rus.txt'

In [4]:
# Vectorize the data.

START = '\t'
END = '\n'

# fill the following variables:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    
    # lowercase
    # append target text with aux tokens

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 50000
Number of unique input tokens: 50
Number of unique output tokens: 78
Max sequence length for inputs: 19
Max sequence length for outputs: 63


In [6]:
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),    dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),    dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),    dtype='float32')

In [7]:
# TODO
# fill encoder_input_data, decoder_input_data, decoder_target_data

## Define model

In [8]:
latent_dim = 256  # Latent dimensionality of the encoding space.

## Decoder model

In [9]:
# Restore the model and construct the encoder and decoder.
model = load_model('char_seq2seq.h5')

encoder_inputs = model.input[0]   # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output   # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]   # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 50)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 78)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 314368      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  343040      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [11]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

In [12]:
# TODO

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = 

    # Generate empty target sequence of length 1.
    # Populate the first character of target sequence with the start character.
    target_seq = 

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        
        # calculate probabilties for next token
        # TODO
        next_char_prob = 
        sampled_token = 
        decoded_sentence += sampled_token

        # Exit condition: either hit max length
        # or find stop token.
        # TODO

        # Update the target sequence (of length 1).
        # TODO

        # Update states
        # TODO

    return decoded_sentence

## BLEU-4 score

![img](BLEU_94.jpg)

In [13]:
from nltk.translate.bleu_score import sentence_bleu


scores = []

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    
    s = sentence_bleu(target_texts[seq_index], decoded_sentence, weights=(0.25, 0.25, 0.25, 0.25))
    scores.append(s)
    
print('mean {} std {}'.format(np.mean(scores), np.std(scores)))

-
Input sentence: go.
Decoded sentence: подождите.

-
Input sentence: go.
Decoded sentence: подождите.

-
Input sentence: hi.
Decoded sentence: вот в порядке.

-
Input sentence: hi.
Decoded sentence: вот в порядке.

-
Input sentence: hi.
Decoded sentence: вот в порядке.

-
Input sentence: hi.
Decoded sentence: вот в порядке.



/opt/anaconda3/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


-
Input sentence: hi.
Decoded sentence: вот в порядке.

-
Input sentence: run!
Decoded sentence: никогда не выиграли.

-
Input sentence: run!
Decoded sentence: никогда не выиграли.

-
Input sentence: run.
Decoded sentence: никогда не выиграть.

-
Input sentence: run.
Decoded sentence: никогда не выиграть.

-
Input sentence: who?
Decoded sentence: кто слышно?

-
Input sentence: wow!
Decoded sentence: подождите!

-
Input sentence: wow!
Decoded sentence: подождите!

-
Input sentence: wow!
Decoded sentence: подождите!

-
Input sentence: wow!
Decoded sentence: подождите!

-
Input sentence: wow!
Decoded sentence: подождите!

-
Input sentence: wow!
Decoded sentence: подождите!

-
Input sentence: fire!
Decoded sentence: положите это!

-
Input sentence: fire!
Decoded sentence: положите это!

-
Input sentence: help!
Decoded sentence: посмотрите!

-
Input sentence: help!
Decoded sentence: посмотрите!

-
Input sentence: help!
Decoded sentence: посмотрите!

-
Input sentence: jump!
Decoded sentence: